In [1]:
!pip install python-dotenv

In [13]:
from dotenv import load_dotenv
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [28]:
# Cargo mis variables de entorno
load_dotenv()

# Genero el dict de datos para conectarme con Snowflake a schema Raw
credencialesSnowflakeAnalytics = {
    "sfURL" : os.getenv("SNOWFLAKE_URL"),
    "sfUser" :  os.getenv("SNOWFLAKE_USER"),
    "sfPassword" : os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase" : os.getenv("SNOWFLAKE_DATABASE"),
    "sfSchema" : os.getenv("SNOWFLAKE_SCHEMA_ANALYTICS"),
    "sfWarehouse" : os.getenv("SNOWFLAKE_WAREHOUSE"),
    "sfRole" : os.getenv("SNOWFLAKE_ROLE"),
}

print(f"Estas son mis credenciales para Snowflake con schema Analytics: {credencialesSnowflakeAnalytics}")

In [29]:
# Creo SparkSession para conexión con Snowflake
spark = (SparkSession.builder.appName("IngestaNewYorkTaxis").config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.30,net.snowflake:spark-snowflake_2.12:2.9.0-spark_3.1").getOrCreate())

print(spark)
print("Spark Version : " + spark.version)

# Ejecuto una query de prueba para validar comunicacion con Snowflake
query = "SELECT current_version()"

df = spark.read.format("snowflake").options(**credencialesSnowflakeAnalytics).option("query", query).load()

df.show()

Spark Version : 3.5.0
+-------------------+
|"CURRENT_VERSION()"|
+-------------------+
|             9.32.1|
+-------------------+



In [30]:
try:
    df_obt_taxis = spark.read.format("snowflake") \
                .options(**credencialesSnowflakeAnalytics) \
                .option("dbtable", "NY_TAXI_OBT_ULTIMATE") \
                .load()
    print("Se cargo correctamente datos de tabla OBT para consultas")
    df_obt_taxis.cache()
except Exception as e:
    print(f"No se pudo cargar tabla OBT:{e}")

Se cargo correctamente datos de tabla OBT para consultas


In [ ]:
#Top 10 zonas de pickup por volumen mensual.

df_top_10_pickup=df_obt_taxis.groupBy("MONTH","PU_ZONE").agg(F.count("*").alias("Viajes_Mensuales_Por_Zona")).orderBy("Viajes_Mensuales_Por_Zona",ascending=False).limit(10)
df_top_10_pickup.show(10)

In [ ]:
#Top 10 zonas de dropoff por volumen mensual.

df_top_10_dropoff=df_obt_taxis.groupBy("MONTH","DO_ZONE").agg(F.count("*").alias("Viajes_Mensuales_Por_Zona")).orderBy("Viajes_Mensuales_Por_Zona",ascending=False).limit(10)
df_top_10_dropoff.show(10)

In [ ]:
#Evolución mensual de total_amount y tip_pct por borough.

df_amount_tip_por_mes=df_obt_taxis.groupBy("MONTH","PU_BOROUGH").agg(F.sum("TOTAL_AMOUNT").alias("Total_Amount_Mensual_Por_Borough"),F.avg("TIP_PCT").alias("Tip_Percentage_Avg_Por_Mes"))
df_amount_tip_por_mes.show(truncate=False)

In [ ]:
#Ticket promedio (avg total_amount) por service_type y mes.

df_ticket_promedio=df_obt_taxis.groupBy("MONTH","SOURCE_SERVICE").agg(F.avg("TOTAL_AMOUNT").alias("Total_Amount_Promedio"))
df_ticket_promedio.show(truncate=False)

In [ ]:
#Viajes por hora del día y día de semana (picos).

df_viajes_hora_dia=df_obt_taxis.groupBy("DAY_OF_WEEK","PICKUP_HOUR").agg(F.count("*").alias("Viajes_Totales"))
df_viajes_hora_dia.show(truncate=False)

In [ ]:
#p50/p90 de trip_duration_min por borough de pickup.

df_percentiles = df_obt_taxis.groupBy("PU_BOROUGH").agg(F.expr('percentile_approx(TRIP_DURATION_MIN, 0.5)').alias('P50_TRIP_DURATION_MIN'),F.expr('percentile_approx(TRIP_DURATION_MIN, 0.9)').alias('P90_TRIP_DURATION_MIN'))
df_percentiles.show(truncate=False)

In [ ]:
#avg_speed_mph por franja horaria (6–9, 17–20) y borough.

df_avg_speed_franja_hora= df_obt_taxis.groupBy("PU_BOROUGH").agg(F.avg(F.when((F.col("PICKUP_HOUR") >= 6) & (F.col("PICKUP_HOUR") <= 9), F.col("AVG_SPEED_MPH"))).alias("AVG_SPEED_MPH_FRANJA_6_9"),
                                                                 F.avg(F.when((F.col("PICKUP_HOUR") >= 17) & (F.col("PICKUP_HOUR") <= 20), F.col("AVG_SPEED_MPH"))).alias("AVG_SPEED_MPH_FRANJA_17_20"))
df_avg_speed_franja_hora.show(truncate=False)                                                               

In [ ]:
#Participación por payment_type_desc y su relación con tip_pct.

df_payment_tip= df_obt_taxis.groupBy("PAYMENT_TYPE_NORMALIZADO").agg(F.avg("TIP_PCT").alias("TIP_PERCENTAGE_PROMEDIO"))
df_payment_tip.show(truncate=False)

In [ ]:
#¿Qué rate_code_desc concentran mayor trip_distance y total_amount?

df_rate_code= df_obt_taxis.groupBy("RATE_CODE_NORMALIZADO").agg(F.sum("TRIP_DISTANCE").alias("TRIP_DISTANCE_ACUMULADO"),F.sum("TOTAL_AMOUNT").alias("TOTAL_AMOUNT_ACUMULADO"))
df_rate_code.show(truncate=False)

In [ ]:
#Mix yellow vs green por mes y borough.

df_yellow_green= df_obt_taxis.groupBy("PU_BOROUGH","SOURCE_SERVICE").agg(F.count("*").alias("VIAJES_TOTALES"))
df_yellow_green.show(truncate=False)

In [ ]:
#Top 20 flujos PU→DO por volumen y su ticket promedio.

df_pu_do= df_obt_taxis.groupBy("PU_LOCATION_ID","DO_LOCATION_ID").agg(F.count("*").alias("VIAJES_TOTALES"),F.avg("TOTAL_AMOUNT").alias("TICKET_PROMEDIO")).orderBy("VIAJES_TOTALES",ascending=False)
df_pu_do.show(20)

In [ ]:
#Distribución de passenger_count y efecto en total_amount.

df_passengers_and_amount= df_obt_taxis.groupBy("PASSENGER_COUNT").agg(F.avg("TOTAL_AMOUNT").alias("TOTAL_AMOUNT_PROMEDIO"))
df_passengers_and_amount.show(truncate=False)

In [ ]:
#Impacto de tolls_amount y congestion_surcharge por zona.

df_impacto_zona = (df_obt_taxis.groupBy("PU_ZONE").agg(F.avg("TOLLS_AMOUNT").alias("TOLLS_AMOUNT_PROMEDIO"),F.avg("CONGESTION_SURCHARGE").alias("CONGESTION_SURCHARGE_PROMEDIO"),
F.avg("TOTAL_AMOUNT").alias("TOTAL_AMOUNT_PROMEDIO")).withColumn("PCT_TOLLS_SOBRE_TOTAL", F.col("TOLLS_AMOUNT_PROMEDIO") / F.col("TOTAL_AMOUNT_PROMEDIO") * 100)
    .withColumn("PCT_CONGESTION_OVER_TOTAL", F.col("CONGESTION_SURCHARGE_PROMEDIO") / F.col("TOTAL_AMOUNT_PROMEDIO") * 100)
    .orderBy(F.desc("TOLLS_AMOUNT_PROMEDIO")))
df_impacto_zona.show(truncate=False)

In [ ]:
#Proporción de viajes cortos vs largos por borough y estacionalidad.

df_proporcion_por_duracion = df_viajes_tipo.groupBy("PU_BOROUGH", "MONTH").agg(F.count("*").alias("CANTIDAD_VIAJES"), F.count(F.when(F.col("TRIP_DURATION_MIN") <= 10, 1)).alias("CANTIDAD_VIAJES_CORTOS"), 
                                                                                F.count(F.when(F.col("TRIP_DURATION_MIN") > 10, 1)).alias("CANTIDAD_VIAJES_LARGOS")).withColumn("PCT_VIAJES_CORTOS", (F.col("CANTIDAD_VIAJES_CORTOS") / F.col("CANTIDAD_VIAJES"))*100).withColumn("PCT_VIAJES_LARGOS", (F.col("CANTIDAD_VIAJES_LARGOS") / F.col("CANTIDAD_VIAJES"))*100) 
df_proporcion_por_duracion.show(truncate=False)

In [ ]:
#Diferencias por vendor en avg_speed_mph y trip_duration_min.

df_vendor_speed_duration= df_obt_taxis.groupBy("VENDOR_NORMALIZADO").agg(F.avg("AVG_SPEED_MPH").alias("SPEED_PROMEDIO"), F.avg("TRIP_DURATION_MIN").alias("TRIP_DURATION_PROMEDIO"))
df_vendor_speed_duration.show(truncate=False)

In [ ]:
#Relación método de pago ↔ tip_amount por hora.

df_pago_tip_amount= df_obt_taxis.groupBy("PAYMENT_TYPE_NORMALIZADO").agg(F.avg("TIP_AMOUNT").alias("TIP_AMOUNT_PROMEDIO"))
df_pago_tip_amount.show(truncate=False)

In [ ]:
#Zonas con percentil 99 de duración/distancia fuera de rango (posible congestión/eventos).

percentiles = df_obt_taxis.select(F.expr("percentile_approx(TRIP_DURATION_MIN, 0.99)").alias("P99_TRIP_DURATION_MIN"),F.expr("percentile_approx(TRIP_DISTANCE, 0.99)").alias("P99_TRIP_DISTANCE")).collect()[0]
p99_duration = percentiles["P99_TRIP_DURATION_MIN"]
p99_distance = percentiles["P99_TRIP_DISTANCE"]
df_fuera_rango = df_obt_taxis.filter((F.col("TRIP_DURATION_MIN") > p99_duration) | (F.col("TRIP_DISTANCE") > p99_distance))
df_zonas_fuera_rango = df_fuera_rango.groupBy("PU_ZONE").agg(F.count(1).alias("CANTIDAD_VIAJES_FUERA_DE_RANGO")).orderBy(F.col("CANTIDAD_VIAJES_FUERA_DE_RANGO").desc())
df_zonas_fuera_rango.show(10, truncate=False)

In [ ]:
#Yield por milla (total_amount/trip_distance) por borough y hora.

df_yield = df_obt_taxis.withColumn("YIELD_POR_MILLA", F.col("TOTAL_AMOUNT") / F.col("TRIP_DISTANCE"))
df_yield_por_borough_hora = (df_yield.groupBy("PU_BOROUGH", "PICKUP_HOUR").agg(F.avg("YIELD_POR_MILLA").alias("YIELD_PROMEDIO_POR_MILLA"),F.count(1).alias("CANTIDAD_VIAJES")))
df_yield_por_borough_hora.show(truncate=False)

In [ ]:
#Cambios YoY en volumen y ticket promedio por service_type.

df_service_year = (df_obt_taxis.groupBy("SOURCE_SERVICE", "YEAR").agg(F.count("*").alias("CANTIDAD_VIAJES"),F.avg("TOTAL_AMOUNT").alias("PROMEDIO_TOTAL_AMOUNT")))
window_spec = Window.partitionBy("SOURCE_SERVICE").orderBy("YEAR")
df_yoy = (df_service_year.withColumn("CAMBIO_VIAJES_YOY", (F.col("CANTIDAD_VIAJES") - F.lag("CANTIDAD_VIAJES").over(window_spec)) / F.lag("CANTIDAD_VIAJES").over(window_spec) * 100).withColumn("CAMBIO_TICKET_YOY", (F.col("PROMEDIO_TOTAL_AMOUNT") - F.lag("PROMEDIO_TOTAL_AMOUNT").over(window_spec)) / F.lag("PROMEDIO_TOTAL_AMOUNT").over(window_spec) * 100))
df_yoy.show(truncate=False)

In [ ]:
#Días con alta congestion_surcharge: efecto en total_amount vs días “normales”

#Se pone valor como umbral
df_congestion = df_obt_taxis.withColumn("TIPO_DIA_CONGESTION",F.when(F.col("CONGESTION_SURCHARGE") > 2.5, F.lit("ALTA")).otherwise(F.lit("NORMAL")))
df_efecto_congestion = (df_congestion.groupBy("TIPO_DIA_CONGESTION").agg( F.count("*").alias("CANTIDAD_VIAJES"),F.avg("TOTAL_AMOUNT").alias("PROMEDIO_TOTAL_AMOUNT")))